# Classiq Coding Competition Spring 2022
### Third Place MCX Submission by Team Carnivorous Cacti 
### Tarushii Goel, Kareem Jaber, Cyril Sharma

### DESCRIPTION

Initially, we solely experimented with the optimal way to apply MCX gates on the ancillary qubits and relied on the qiskit transpiler to determine the circuit depth when MCX gates were decomposed into single qubit gates and CX gates. Our main strategy at this stage was to minimize the overlap between the bits used in the MCX gates, since MCX gates which don’t overlap can be executed simultaneously and have lower circuit depth. With this strategy we got a depth of 151 (see additional files for the approaches we tried, design #2 for our best approach). One observation we made is that assuming that all the ancillary bits are used, you will be used a total of 19 controls across 6 targets, so you must use at least one MCX gate with 4 controls (6*3 < 19). Optimizing the 4 control MCX gate was our main bottleneck. To improve our initial solutions we realized that the many of the MCX gates only had to be correct up to a relative phase, since we can remove the phase shift while uncomputing (https://arxiv.org/pdf/1508.03273.pdf). We used the relative phase toffoli-4 introduced in this paper and extended it to a relative phase toffoli-5 (see the function Rc4x()) and substituted the MCX gates (which in our previous implementation had 3 or 4 controls) with the relative phase versions. This approach achieved a final circuit depth of 71.

In [1]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary
import qiskit
from scipy.linalg import norm

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# relative phase toffoli gate
from qiskit.circuit.library import RC3XGate


In [2]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')

In [3]:
def Rc4x():
    # relative phase toffoli-5 implementation
    qc = QuantumCircuit(5)
    qc.h(4)
    qc.t(4)
    qc.cx(3, 4)
    qc.tdg(4)
    qc.h(4)
    qc.mcx([0, 1], 4)
    qc.h(4)
    qc.append(RC3XGate(), [0, 1, 2, 4])
    # qc.mcx([0, 1, 2], 4)
    qc.t(4)
    qc.cx(3, 4)
    qc.tdg(4)
    qc.h(4)

    rc4x = qc.to_gate()
    rc4x.name = "Rc4x"
    return rc4x

qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)

# qc.h(control_qubits[:14])
# qc.x(target_qubit)

for i in range(3):
    qc.append(RC3XGate(), control_qubits[3*i:3*(i+1)]+ancilla_qubits[i:i+1])
# 0 - 8
qc.append(Rc4x(), control_qubits[9:13]+ancilla_qubits[3:4])

qc.append(RC3XGate(), ancilla_qubits[0:2] + control_qubits[13:14] + ancilla_qubits[4:5])

qc.mcx(ancilla_qubits[2:5], target_qubit[0])

# # uncompute
qc.append(RC3XGate().inverse(), ancilla_qubits[0:2] + control_qubits[13:14] + ancilla_qubits[4:5])
for i in range(3):
    qc.append(RC3XGate().inverse(), control_qubits[3*i:3*(i+1)]+ancilla_qubits[i:i+1])
# # 0 - 8
qc.append(Rc4x().inverse(), control_qubits[9:13]+ancilla_qubits[3:4])

#qc.measure(target_qubit[0], classical_bits[0])

qc.draw()

┌────────┐                                                         »
  c_0: ┤0       ├─────────────────────────────────────────────────────────»
       │        │                                                         »
  c_1: ┤1       ├─────────────────────────────────────────────────────────»
       │        │                                                         »
  c_2: ┤2       ├─────────────────────────────────────────────────────────»
       │        │┌────────┐                                               »
  c_3: ┤        ├┤0       ├───────────────────────────────────────────────»
       │        ││        │                                               »
  c_4: ┤        ├┤1       ├───────────────────────────────────────────────»
       │        ││        │                                               »
  c_5: ┤        ├┤2       ├───────────────────────────────────────────────»
       │        ││        │┌────────┐                                     »
  c_6: ┤        ├┤        ├┤0       ├─────────────────────────────────────»
       │        ││        ││        │                                     »
  c_7: ┤        ├┤        ├┤1       ├─────────────────────────────────────»
       │  Rcccx ││        ││        │                                     »
  c_8: ┤        ├┤        ├┤2       ├─────────────────────────────────────»
       │        ││        ││        │┌───────┐                            »
  c_9: ┤        ├┤        ├┤        ├┤0      ├────────────────────────────»
       │        ││  Rcccx ││        ││       │                            »
 c_10: ┤        ├┤        ├┤        ├┤1      ├────────────────────────────»
       │        ││        ││        ││       │                            »
 c_11: ┤        ├┤        ├┤        ├┤2      ├────────────────────────────»
       │        ││        ││  Rcccx ││       │                            »
 c_12: ┤        ├┤        ├┤        ├┤3      ├────────────────────────────»
       │        ││        ││        ││       │┌────────┐     ┌───────────┐»
 c_13: ┤        ├┤        ├┤        ├┤       ├┤2       ├─────┤2          ├»
       │        ││        ││        ││  Rc4x ││        │┌───┐│           │»
    t: ┤        ├┤        ├┤        ├┤       ├┤        ├┤ X ├┤           ├»
       │        ││        ││        ││       ││        │└─┬─┘│           │»
  a_0: ┤3       ├┤        ├┤        ├┤       ├┤0       ├──┼──┤0          ├»
       └────────┘│        ││        ││       ││        │  │  │           │»
  a_1: ──────────┤3       ├┤        ├┤       ├┤1 Rcccx ├──┼──┤1 rcccx_dg ├»
                 └────────┘│        ││       ││        │  │  │           │»
  a_2: ────────────────────┤3       ├┤       ├┤        ├──■──┤           ├»
                           └────────┘│       ││        │  │  │           │»
  a_3: ──────────────────────────────┤4      ├┤        ├──■──┤           ├»
                                     └───────┘│        │  │  │           │»
  a_4: ───────────────────────────────────────┤3       ├──■──┤3          ├»
                                              └────────┘     └───────────┘»
out: 1/═══════════════════════════════════════════════════════════════════»
                                                                          »
«                                ┌───────────┐             
«  c_0: ─────────────────────────┤0          ├─────────────
«                                │           │             
«  c_1: ─────────────────────────┤1          ├─────────────
«                                │           │             
«  c_2: ─────────────────────────┤2          ├─────────────
«                                │           │┌───────────┐
«  c_3: ─────────────────────────┤           ├┤0          ├
«                                │           ││           │
«  c_4: ─────────────────────────┤           ├┤1          ├
«                                │           ││           │
«  c_5: ─────────────────────────┤           ├┤2          ├
«       ┌──────────

In [4]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())
qc_transpiled.qasm(filename='toffoli1.qasm')

71
OrderedDict([('u', 101), ('cx', 90)])


'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg c[14];\nqreg t[1];\nqreg a[5];\ncreg out[1];\nu(pi/2,pi/8,-pi) t[0];\nu(pi/2,pi/4,-pi) a[0];\ncx c[2],a[0];\nu(pi/2,0,3*pi/4) a[0];\ncx c[0],a[0];\nu(0,0,pi/4) a[0];\ncx c[1],a[0];\nu(0,0,-pi/4) a[0];\ncx c[0],a[0];\nu(0,0,pi/4) a[0];\ncx c[1],a[0];\nu(pi/2,pi/4,3*pi/4) a[0];\ncx c[2],a[0];\nu(pi/2,0,3*pi/4) a[0];\nu(pi/2,pi/4,-pi) a[1];\ncx c[5],a[1];\nu(pi/2,0,3*pi/4) a[1];\ncx c[3],a[1];\nu(0,0,pi/4) a[1];\ncx c[4],a[1];\nu(0,0,-pi/4) a[1];\ncx c[3],a[1];\nu(0,0,pi/4) a[1];\ncx c[4],a[1];\nu(pi/2,pi/4,3*pi/4) a[1];\ncx c[5],a[1];\nu(pi/2,0,3*pi/4) a[1];\nu(pi/2,pi/4,-pi) a[2];\ncx c[8],a[2];\nu(pi/2,0,3*pi/4) a[2];\ncx c[6],a[2];\nu(0,0,pi/4) a[2];\ncx c[7],a[2];\nu(0,0,-pi/4) a[2];\ncx c[6],a[2];\nu(0,0,pi/4) a[2];\ncx c[7],a[2];\nu(pi/2,pi/4,3*pi/4) a[2];\ncx c[8],a[2];\nu(pi/2,pi/8,3*pi/4) a[2];\nu(pi/2,pi/4,-pi) a[3];\ncx c[12],a[3];\nu(0,2.1003505,-2.8857486) a[3];\ncx c[10],a[3];\nu(0,0,-pi/4) a[3];\ncx c[9],a[3];\nu(0,0,pi/4) a[3];\

In [5]:
state = qiskit.quantum_info.Statevector.from_instruction(qc)
for elem in state.to_dict(decimals=3).values():
    if elem - 0.007812499999999984+4.293221566713447e-18j > 0.1:
        print(elem)

(0.9999999999999972+4.921270418292891e-16j)


In [6]:
# test efficiency of various sizes with no-ancilla
numControls = 10
modes = ['noancilla', 'recursion', 'v-chain', 'v-chain-dirty']
for m in modes:
    print(f"Mode: {m}")
    for i in range(1, numControls):
        ancilla_qubits = QuantumRegister(15, name='a')
        control_qubits = QuantumRegister(i, name="c")
        qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
        for j in range(i):
            qc.x(control_qubits[j])
        qc.mcx(control_qubits, ancilla_qubits[0], ancilla_qubits=ancilla_qubits[1:], mode=m)
        qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
        if (m == 'noancilla'):
            num_ancilla = 0
        elif (m == 'recursion'):
            num_ancilla = 0 if i < 5 else 1
        elif (m == 'v-chain' or m =='v-chain-dirty'):
            num_ancilla = i - 2 if i > 2 else 0
        print(f"Controls: {i}, Depth: {qc_transpiled.depth()}, Ancilla: {num_ancilla}, Ops: {qc_transpiled.count_ops()}")
    print()

Mode: noancilla
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65, Ancilla: 0, Ops: OrderedDict([('u', 44), ('cx', 36)])
Controls: 5, Depth: 155, Ancilla: 0, Ops: OrderedDict([('u', 98), ('cx', 92)])
Controls: 6, Depth: 315, Ancilla: 0, Ops: OrderedDict([('u', 195), ('cx', 188)])
Controls: 7, Depth: 635, Ancilla: 0, Ops: OrderedDict([('u', 388), ('cx', 380)])
Controls: 8, Depth: 1275, Ancilla: 0, Ops: OrderedDict([('u', 773), ('cx', 764)])
Controls: 9, Depth: 2555, Ancilla: 0, Ops: OrderedDict([('u', 1542), ('cx', 1532)])

Mode: recursion
Controls: 1, Depth: 2, Ancilla: 0, Ops: OrderedDict([('u', 1), ('cx', 1)])
Controls: 2, Depth: 11, Ancilla: 0, Ops: OrderedDict([('u', 10), ('cx', 6)])
Controls: 3, Depth: 27, Ancilla: 0, Ops: OrderedDict([('u', 16), ('cx', 14)])
Controls: 4, Depth: 65

In [7]:
# Testing my gate
def Rc4x():
    # relative phase toffoli-5 implementation
    qc = QuantumCircuit(5)
    qc.h(4)
    qc.t(4)
    qc.cx(3, 4)
    qc.tdg(4)
    qc.h(4)
    qc.mcx([0, 1], 4)
    qc.h(4)
    qc.append(RC3XGate(), [0, 1, 2, 4])
    # qc.mcx([0, 1, 2], 4)
    qc.t(4)
    qc.cx(3, 4)
    qc.tdg(4)
    qc.h(4)

    rc4x = qc.to_gate()
    rc4x.name = "Rc4x"
    return rc4x

qc = QuantumCircuit(6)
qc.append(Rc4x(), range(5))
qc.cx(4, 5)
qc.append(Rc4x().inverse(), range(5))
#circuit already defined
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
U_circuit = result.get_unitary(qc, decimals=3)
qc.draw()

┌───────┐     ┌──────────┐
q_0: ┤0      ├─────┤0         ├
     │       │     │          │
q_1: ┤1      ├─────┤1         ├
     │       │     │          │
q_2: ┤2 Rc4x ├─────┤2 Rc4x_dg ├
     │       │     │          │
q_3: ┤3      ├─────┤3         ├
     │       │     │          │
q_4: ┤4      ├──■──┤4         ├
     └───────┘┌─┴─┐└──────────┘
q_5: ─────────┤ X ├────────────
              └───┘

In [8]:
qc = QuantumCircuit(6)
qc.mcx(list(range(4)), 5)
qc.cx(4, 5)
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
U_target = result.get_unitary(qc)
qc.draw()

q_0: ──■───────
       │       
q_1: ──■───────
       │       
q_2: ──■───────
       │       
q_3: ──■───────
       │       
q_4: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_5: ┤ X ├┤ X ├
     └───┘└───┘

In [9]:
norm(U_circuit-U_target) # 0 indicates that the circuit works correctly and matches the target matrix

0.0

In [10]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'], optimization_level=3)
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

66
OrderedDict([('u', 41), ('cx', 37)])
